In [1]:
import gdown
import pandas as pd
import os
import csv
import re

In [2]:
def download_csv():
    sheet_url = "https://docs.google.com/spreadsheets/d/1vYD9nyEoI2CEWu2UY2xpedC_4mX3TUVD-XLCjOLphiA/export?format=csv"

    # Load the CSV into a DataFrame
    df = pd.read_csv(sheet_url)
    df.to_csv("form.csv")
    # print(df)

In [3]:
download_csv()

In [4]:
original_csv = "form.csv"
processed_csv = "processed.csv"
clothes_folder = "clothes_data"
possible_columns = [
    "服裝部位 Clothing Part",
    "上半身 衣服種類 Top Clothing Type",
    "下半身衣服種類 Bottom Clothing Type",
]

In [5]:
df = pd.read_csv(original_csv)
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df2 = df.drop("時間戳記", axis=1)
for col in possible_columns:
    df2 = df2.drop(col, axis=1)

In [18]:
# get all feature
all_features = []
for index, row in df.iterrows():
    for col in possible_columns:
        str = df.loc[index, col]
        try:
            str = list(str.split(","))
            for s in str:
                s = s.rstrip()
                s = s.lstrip()
                if (
                    s == "針織衣 Sweater"
                    or s == "毛衣 Sweater"
                    or s == "針/織衣 Sweater"
                    or s == "毛衣、針/織衣 Sweater"
                ):
                    s = "毛衣、針織衣 Sweater"
                if s == "西裝褲 dress pants":
                    str.append("褲子 pants")
                    s = "西裝褲、裙 dress pants、skirt"
                all_features.append(s)
        except:
            pass
all_features = set(all_features)
for f in all_features:
    print(f)
print(len(all_features))

上半身 Top
裙子 skirt
運動褲/裙 sweat pants / sports skirt
羽絨衣 Down coat / jacket
下半身 Bottom
網紗 Mesh
百褶裙 pleated skirt
Shirt
工裝褲/裙 cargo pants/skirt
連帽衣 Hoodie
吊帶褲/裙 overalls/jumper skirt
西裝褲、裙 dress pants、skirt
洋裝 Dress
牛仔褲/裙 jeans / demi skirt
外套 Jacket
牛仔材質 Denim
毛衣、針織衣 Sweater
運動材質 Sport Tops
西裝 Suit
緊身褲 leggings
休閒褲/裙 casual pants/skirt
褲子 pants
T-Shirt
背心 Vest
24


In [19]:
for index, row in df.iterrows():
    for fea in all_features:
        has = 0
        for col in possible_columns:
            str = df.loc[index, col]
            try:
                str = list(str.split(","))
                for s in str:
                    s = s.rstrip()
                    s = s.lstrip()
                    if (
                        s == "針織衣 Sweater"
                        or s == "毛衣 Sweater"
                        or s == "針/織衣 Sweater"
                        or s == "毛衣、針/織衣 Sweater"
                    ):
                        s = "毛衣、針織衣 Sweater"
                    if s == "西裝褲 dress pants":
                        str.append("褲子 pants")
                        s = "西裝褲、裙 dress pants、skirt"
                    if s == fea:
                        has = 1
            except:
                pass
        df2.loc[index, fea] = has


print(df2)

           照片 Picture      品牌 Brand 衣服長度 Clothes Length  價格 Price  上半身 Top  \
0     image_00000.jpg           H&M             短 short       499      1.0   
1     image_00001.jpg           H&M              長 long       999      1.0   
2     image_00002.jpg           H&M              長 long       699      0.0   
3     image_00003.jpg           H&M              長 long       999      0.0   
4     image_00004.jpg           H&M              長 long       699      0.0   
...               ...           ...                 ...       ...      ...   
1805  image_01805.jpg          Nike              長 long      2180      1.0   
1806  image_01806.jpg          Nike              長 long      2180      1.0   
1807  image_01807.jpg          Nike              長 long      2180      1.0   
1808  image_01808.jpg  Under Armour              長 long      2680      0.0   
1809  image_01809.jpg  Under Armour              長 long      2080      0.0   

      裙子 skirt  運動褲/裙 sweat pants / sports skirt  毛衣、針/織衣 Sweat

In [20]:
for fea in all_features:
    has = 0
    for index, row in df.iterrows():
        for col in possible_columns:
            str = df.loc[index, col]
            try:
                str = list(str.split(","))
                for s in str:
                    s = s.rstrip()
                    s = s.lstrip()
                    if (
                        s == "針織衣 Sweater"
                        or s == "毛衣 Sweater"
                        or s == "針/織衣 Sweater"
                        or s == "毛衣、針/織衣 Sweater"
                    ):
                        s = "毛衣、針織衣 Sweater"
                    if s == "西裝褲 dress pants":
                        str.append("褲子 pants")
                        s = "西裝褲、裙 dress pants、skirt"
                    if s == fea:
                        has += 1
            except:
                pass
    print(fea + ":", has)

上半身 Top: 1171
裙子 skirt: 15
運動褲/裙 sweat pants / sports skirt: 123
羽絨衣 Down coat / jacket: 42
下半身 Bottom: 522
網紗 Mesh: 11
百褶裙 pleated skirt: 37
Shirt: 186
工裝褲/裙 cargo pants/skirt: 34
連帽衣 Hoodie: 142
吊帶褲/裙 overalls/jumper skirt: 5
西裝褲、裙 dress pants、skirt: 80
洋裝 Dress: 117
牛仔褲/裙 jeans / demi skirt: 76
外套 Jacket: 234
牛仔材質 Denim: 2
毛衣、針織衣 Sweater: 203
運動材質 Sport Tops: 165
西裝 Suit: 22
緊身褲 leggings: 26
休閒褲/裙 casual pants/skirt: 210
褲子 pants: 139
T-Shirt: 431
背心 Vest: 155


In [8]:
for index, row in df2.iterrows():
    image_url = row["照片 Picture"]  # 從表單中的照片 URL 讀取

    if re.search(r"open\?id=", image_url):
        output = f"image_{index:05}.jpg"
        df2.loc[index, "照片 Picture"] = output
df2.to_csv(processed_csv)

# check image downloaded or not

In [9]:
def download_image(image_url, path):
    file_id = image_url.split("open?id=")[-1]
    url = f"https://drive.google.com/uc?export=download&id={file_id}"
    gdown.download(url, path, quiet=False)

In [10]:
for index, row in df2.iterrows():
    output = f"image_{index:05}.jpg"
    path = os.path.join(clothes_folder, output)
    if not os.path.exists(path):
        image_url = df.loc[index, "照片 Picture"]
        print("download ", index, " from ", image_url)
        download_image(image_url, path)

download  1040  from  https://drive.google.com/open?id=1Oac6kidpmUBnZJF23ou9WYUybD74Q_Z7


Downloading...
From: https://drive.google.com/uc?export=download&id=1Oac6kidpmUBnZJF23ou9WYUybD74Q_Z7
To: c:\Users\doo12\OneDrive\桌面\ML\finalProject\ClothesPredicting\clothes_data\image_01040.jpg
100%|██████████| 133k/133k [00:00<00:00, 65.8MB/s]


download  1041  from  https://drive.google.com/open?id=1auPHpNK0ldlPXxOeLb7_wSnuQ-FsgJYT


KeyboardInterrupt: 